# Diseccionando una red neuronal 

**Autor:**
> Profesor Jorge Anais Vilchez  
> Abril 2024  

En este notebook se muestra como programar una red neuronal a un nivel menos abstracto de lo que hemos revisado, a fin de que podemas ilustrar de mejor manera el funcionamiento de esta. Para ello utilizaremos Pytorch, una librería de Python que nos permite trabajar con redes neuronales de manera sencilla.

Finalmente se muestra una manera más sencilla y usual de programar una red neuronal utilizando Pytorch, es decir, algunos detalles de la implementación quedan ocultos.

Este material se basa en la clases del profesor Jorge Pérez publicada en YT (https://youtu.be/y6aD4WG-rOw?si=fyNnilzbXLdWPFyr)

In [1]:
import torch
import numpy as np
import sys
import time

# Fijamos una semilla para que los experimentos sean reproducibles
t_cg = torch.manual_seed(1234)

## Definición de algunas funciones de activación y función de error, y la respectiva derivada

Sigmoide:

$$\sigma(t) = \frac{1}{1 + e^{-t}}; \quad \quad \quad \frac{d}{dt} \sigma(t)= \sigma(t)(1-\sigma(t))$$

Tangente hiperbólica:

$$\tanh(t) = \frac{e^t - e^{-t}}{e^t + e^{-t}}; \quad \quad \quad \frac{d}{dt} \tanh(t)= 1 - \tanh^2(t)$$

Entropía cruzada binaria:

$$ \mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y_i}) - (1 - y_i) \log(1 - \hat{y_i}) \right] \quad \quad \quad \frac{\partial}{\partial\hat{y}} \mathcal{L} = \frac{1}{N}(\hat{y} - y)$$

In [2]:
def sig(T: torch.Tensor):
    """Función de activación sigmoide"""
    return torch.reciprocal(1 + torch.exp(-1 * T))


def tanh(T: torch.Tensor):
    """Función de activación tangente hiperbólica."""
    E = torch.exp(T)
    e = torch.exp(-1 * T)
    return (E - e) * torch.reciprocal(E + e)


def bi_cross_ent_loss(y_pred, y, safe=True, epsilon=1e-7):
    """Función de pérdida de entropía cruzada binaria"""
    
    N = y.size()[0]  # tamaño del batch
    
    # Asegura que no haya valores indefinidos.
    if safe:
        y_pred = y_pred.clamp(epsilon, 1 - epsilon) 
    
    B = (1-y) * torch.log(1 - y_pred) + y * torch.log(y_pred)
    return -1/N * torch.sum(B)  

## Definición de la red neuronal

Consideremos la siguiente red con dos capas ocultas
![Imagen](./imagenes/red_manual/esquema.png)

Escribimos el paso de foward y backward (aquí necesitamos también las derivadas)

In [3]:
class FFNN(torch.nn.Module):
    def __init__(self, d0=300, d1=200, d2=300, init_v=1):
        """
        Crea la red FFNN con 2 capas ocultas y una capa de salida.
        d0: dimensión de la capa de entrada
        d1: dimensión de la primera capa oculta
        d2: dimensión de la segunda capa oculta
        init_v: multiplicador valor inicial de los pesos
        """
        super(FFNN, self).__init__()
        
        # Crea los tensores como parámetros
        self.W1 = torch.nn.Parameter(torch.randn(d0,d1) * init_v) 
        self.b1 = torch.nn.Parameter(torch.zeros(d1))
        self.W2 = torch.nn.Parameter(torch.randn(d1,d2) * init_v)
        self.b2 = torch.nn.Parameter(torch.zeros(d2))
        self.U  = torch.nn.Parameter(torch.randn(d2,1) * init_v)
        self.c  = torch.nn.Parameter(torch.zeros(1))
            
    
    def forward(self, x: torch.Tensor):
        # Calcula la pasada hacia adelante
        u1 = x @ self.W1 + self.b1
        h1 = tanh(u1)
        u2 = h1 @ self.W2 + self.b2
        h2 = sig(u2)
        u3 = h2 @ self.U + self.c
        y_pred = sig(u3)
        
        self._cache = [u1, u2]  # Guaradamos temporalmente los valores de u1 y u2
        
        return y_pred
    
    # Backpropagation 
    def backward(self, x: torch.Tensor, y: torch.Tensor, y_pred: torch.Tensor):
        
        u1, u2 = self._cache  # recuperamos los valores de u1 y u2

        # tamaño del batch
        b = x.size()[0]
        
        # Estas son derivadas calculadas a mano
        dL_du3 = (1/b) * (y_pred - y)
        dL_dU  = sig(u2).t() @ dL_du3
        dL_dc  = torch.sum(dL_du3, 0)
        dL_dh2 = dL_du3 @ self.U.t()
        dL_du2 = dL_dh2 * sig(u2) * (1 - sig(u2)) 
        dL_dW2 = tanh(u1).t() @ dL_du2
        dL_db2 = torch.sum(dL_du2, 0)
        dL_dh1 = dL_du2 @ self.W2.t()
        dL_du1 = dL_dh1 * (1 - tanh(u1) * tanh(u1))
        dL_dW1 = x.t() @ dL_du1
        dL_db1 = torch.sum(dL_du1, 0)
                
        # Registra los valores de gradientes en cada tensor (que nos interesa)
        grads = [dL_dW1, dL_db1, dL_dW2, dL_db2, dL_dU, dL_dc]
        params = [self.W1, self.b1, self.W2, self.b2, self.U, self.c]
        for p, g in zip(params, grads):
            p.grad = g 
            
    def num_parameters(self):
        total = 0
        for p in self.parameters():
            total += p.numel()
        return total

## Preparación de Datos

Aquí se genera un conjunto de datos aleatorios para entrenar la red neuronal. La clase `Dataset` define un conjunto de datos y la clase `DataLoader` permite cargar los datos en lotes.
Es una manera muy eficiente, puede cargar en paralelo en la CPU y GPU.

In [4]:
from torch.utils.data import Dataset, DataLoader

class RandomDataSet(Dataset):
    def __init__(self, N: int, f: int):
        """Genera datos al azar. N indica el numero de ejemplos y f el número de caracteristicas (features)"""
        R_N_f = torch.rand(N,f)
        self.X = torch.bernoulli(R_N_f)
        R_N_1 = torch.rand(N,1)
        self.Y = torch.bernoulli(R_N_1)
        self.num_features = f
        
    # Debemos definir __len__ para retornar el tamaño del dataset
    def __len__(self):
        return self.X.size()[0]

    # Debemos definir __getitem__ para retornar el i-ésimo 
    # ejemplo en nuestro dataset.
    def __getitem__(self, i):
        return self.X[i], self.Y[i]

## Bucle de entrenamiento

Ahora debemos utilizar nuestros datos para entrenar la red. Para ello, necesitamos un bucle que haga lo siguiente:

1. Tomar un lote o subconjunto de nuestros datos de entrenamiento
2. Calcular la salida de la red (forward).
3. Calcular la función de error.
4. Calcular las derivadas de los pesos (backward).
5. Actualizar los pesos utilizando el descenso del gradiente.



In [5]:
def loop_FFNN(
    dataset: Dataset,
    batch_size: int,   # tamaño del lote
    d1: int,  # número de neuronas en la capa 1
    d2: int,  # número de neuronas en la capa 2
    lr: float,  # tasa de aprendizaje
    N: int, # numero de ejemplos
    epochs: int,
    run_in_GPU: bool=True,
    reports_every: int=1,
    init_v: float = 1.,
):
    
    # Define un tipo para los tensores según si correrá en la GPU o no
    device = 'cuda' if run_in_GPU else 'cpu'
        
    # d0 es la cantidad de features    
    d0 = dataset.num_features
    
    # Crea la red
    red = FFNN(d0, d1, d2, init_v)
        
    # Pasa la red al dispositivo elegido
    red.to(device)
    
    # Muestra la cantidad de parámetros
    print(f"Cantidad de parámetros: {red.num_parameters()}")
    
    # Crea un dataloader desde el dataset
    data = DataLoader(dataset, batch_size, shuffle=True)
    
    # Comienza el entrenamiento
    tiempo_epochs = 0
    for e in range(1, epochs + 1):    
        inicio_epoch = time.process_time()
        
        for (x, y) in data:         
            # Asegura de pasarlos a la GPU si fuera necesario
            x, y = x.to(device), y.to(device)
            
            # Computa la pasada hacia adelante (forward)
            y_pred = red.forward(x)
            
            # Computa la función de pérdida
            L = bi_cross_ent_loss(y_pred, y)
            
            # Computa los gradientes hacia atrás (backpropagation)
            red.backward(x, y, y_pred)
            
            
            # Descenso de gradiente para actualizar los parámetros
            for p in red.parameters():
                p.data = p.data - lr * p.grad
            
        tiempo_epochs += time.process_time() - inicio_epoch
        
        # Reporta el acierto cada "reports_every" cantidad de épocas
        if e % reports_every == 0:
            
            # Calcula la certeza de las predicciones sobre todo el conjunto
            X = dataset.X.to(device)
            Y = dataset.Y.to(device)

            # Predice usando la red
            Y_PRED = red.forward(X)
            
            # Calcula la pérdida de todo el conjunto
            L_total = bi_cross_ent_loss(Y_PRED, Y)

            # Elige una clase dependiendo del valor de Y_PRED
            Y_PRED_BIN = (Y_PRED >= 0.5).float()

            correctos = torch.sum(Y_PRED_BIN == Y).item()
            acc = (correctos / N) * 100

            sys.stdout.write(
                f"Epoch:{e:03d} Acc:{acc:.2f} Loss:{L_total:.4f} Tiempo/epoch:{tiempo_epochs/e:.3f}s\n"
            )

# Ejemplo de como ejecutar nuestra red

In [6]:
N = 5000 # numero de ejemplos
f = 300 # numero de features

dataset = RandomDataSet(N,f)

In [7]:
epochs = 20

loop_FFNN(
    dataset,
    batch_size=64,  # 10
    d1=300,
    d2=400,
    N=N,
    epochs=epochs,
    run_in_GPU=True,
    lr=0.06,
    init_v=1.,
)

Cantidad de parámetros: 211101
Epoch:001 Acc:56.16 Loss:2.5187 Tiempo/epoch:0.155s


Epoch:002 Acc:59.92 Loss:1.9686 Tiempo/epoch:0.133s
Epoch:003 Acc:67.20 Loss:1.3564 Tiempo/epoch:0.126s
Epoch:004 Acc:66.58 Loss:1.2281 Tiempo/epoch:0.122s
Epoch:005 Acc:76.20 Loss:0.7849 Tiempo/epoch:0.119s
Epoch:006 Acc:73.78 Loss:0.7616 Tiempo/epoch:0.117s
Epoch:007 Acc:88.18 Loss:0.4127 Tiempo/epoch:0.116s
Epoch:008 Acc:90.40 Loss:0.3360 Tiempo/epoch:0.114s
Epoch:009 Acc:92.70 Loss:0.2717 Tiempo/epoch:0.114s
Epoch:010 Acc:94.30 Loss:0.2196 Tiempo/epoch:0.113s
Epoch:011 Acc:93.32 Loss:0.2264 Tiempo/epoch:0.112s
Epoch:012 Acc:95.28 Loss:0.1769 Tiempo/epoch:0.112s
Epoch:013 Acc:97.00 Loss:0.1309 Tiempo/epoch:0.112s
Epoch:014 Acc:97.22 Loss:0.1302 Tiempo/epoch:0.113s
Epoch:015 Acc:98.08 Loss:0.1020 Tiempo/epoch:0.115s
Epoch:016 Acc:98.52 Loss:0.0894 Tiempo/epoch:0.115s
Epoch:017 Acc:98.76 Loss:0.0802 Tiempo/epoch:0.114s
Epoch:018 Acc:99.04 Loss:0.0711 Tiempo/epoch:0.114s
Epoch:019 Acc:98.98 Loss:0.0729 Tiempo/epoch:0.114s
Epoch:020 Acc:99.24 Loss:0.0595 Tiempo/epoch:0.114s


## Preguntas

1. **¿Qué pasa al aumentar el tamaño del lote (`batch_size`) de 10 a 64?**   
R: El trabajo total se mantiene constante (la cantidad de multiplicaciones es la misma). El tiempo de ejecución disminuye porque se procesan de manera más óptima ("la red está mirando más valores al mismo tiempo").

2. **Analice el tiempo por epoca en los siguientes casos**:

|              | CPU    | GPU    | CPU     | GPU     | CPU     | GPU     |
|--------------|--------|--------|---------|---------|---------|---------|
| epochs       | 20     | 20     | 20      | 20      | 20      | 20      |
| batch size   | 128    | 128    | 128     | 128     | 128     | 128     |
| d1           | 500    | 500    | 1000    | 1000    | 2000    | 2000    |
| d2           | 800    | 800    | 2000    | 2000    | 2000    | 2000    |
| tiempo/epoca | 0.45s  | 0.10s  | 4.45s   | 0.10s   | 19.22s  | 0.10s   |
| N params     | 552101 | 552101 | 2305001 | 2305001 | 4606001 | 4606001 |
| acc          | 99.38% | 99.30% | --      | --      | --      | --      |

La mejora en tiempo que logra la CPU con respecto a la CPU es más evidente cuando la red es más grande. Esto se debe a que hay un tiempo fijo de carga de datos entre la CPU y la GPU.

3. **Los parámetros se inicializan con una distribución normal estándar, es decir con valores alrededor de cero con una desviación estándar de 1. ¿Qué pasa agrandamos estos números (`init_v=2`)?**   
R: notar que la acc baja (~50%) y la función de error se dispara (nan). Notar que tenemos multiplicaciones de matrices que se multiplican por matrices. Esto es muy sensible cuando los números de los parámetros son más grandes que 1, y podrían crecer exponencialmente. Esto podría causar que la red sea inestable. Por otra parte, si se usan numeros muy chiquitos (`init_v=0.001`) la red no es capaz de aprender y su accuracy se queda cercana a ~50%.

4. **¿Qué pasa si aumentamos el tamaño del batch y dejamos la cantidad de épocas fijas?**  
R: La cantidad de veces que realizas el "descenso del grandiente" es menor. 

5. **Comentario: En vez de usar red.backward(), podemos dejar que pytorch lo haga por nosotros, incluyendo el cálculo de las derivadas.**
Comentar la línea `red.backward(x, y, y_pred)` y reemplazar por
```python
for p in red.parameters():
    if p.grad is not None:
        p.grad.zero_()

L.backward()
```

# Red neuronal estilo Pytorch

Pytorch ya implementa varias capas de redes predefinidas. Por ejemplo, las redes fully connected están implementadas en `torch.nn.Linear`. Además, Pytorch ya implementa varias funciones de activación, como la sigmoide, tangente hiperbólica, etc.

Al hacerlo de esta manera, pytorch crea los parámetros de manera automática y los inicializa con buenas heurísticas.

In [8]:
# Red estilo pytorch
class FFNN(torch.nn.Module):
    def __init__(self, d0: int = 300, d1: int = 200, d2: int = 300):
        super(FFNN, self).__init__()
        
        # Definimos capas (automáticamente se registran como parametros)
        self.fc1 = torch.nn.Linear(d0, d1, bias=True)
        self.fc2 = torch.nn.Linear(d1, d2, bias=True)
        self.fc3 = torch.nn.Linear(d2,  1, bias=True)
    
    # Computa la pasada hacia adelante
    def forward(self, x: torch.Tensor):
    
        u1 = self.fc1(x)
        h1 = torch.tanh(u1)
        u2 = self.fc2(h1)
        h2 = torch.sigmoid(u2)
        u3 = self.fc3(h2)
        y_pred = torch.sigmoid(u3)
            
        return y_pred

In [9]:
def loop_FFNN_pytorch_style(
    dataset: Dataset,
    batch_size: int,   # tamaño del lote
    d1: int,  # número de neuronas en la capa 1
    d2: int,  # número de neuronas en la capa 2
    lr: float,  # tasa de aprendizaje
    N: int, # numero de ejemplos
    epochs: int,
    run_in_GPU: bool=True,
    reports_every: int=1,
):
    
    # Define un tipo para los tensores según si correrá en la GPU o no
    device = 'cuda' if run_in_GPU else 'cpu'
        
    # d0 es la cantidad de características (features)    
    d0 = dataset.num_features
    
    # Crea la red
    red = FFNN(d0, d1, d2)
        
    # Pasa la red al dispositivo elegido
    red.to(device)
                
    # Muestra la cantidad de parámetros
    print('Número de parámetros de la red:', red)
    
    # Crea un dataloader desde el dataset
    data = DataLoader(dataset, batch_size, shuffle=True)
    
    # Crea un optimizador para el descenso de gradiente
    optimizador = torch.optim.SGD(red.parameters(), lr)
    
    # Define una perdida
    perdida = torch.nn.BCELoss()
    
    # Comienza el entrenamiento
    tiempo_epochs = 0
    for e in range(1,epochs+1):    
        inicio_epoch = time.process_time()
        
        for (x,y) in data:         
            # Asegura de pasarlos a la GPU si fuera necesario
            x, y = x.to(device), y.to(device)
            
            # Computa la pasada hacia adelante (forward)
            y_pred = red.forward(x)
            
            # Computa la función de pérdida
            L = perdida(y_pred,y)
            
            # Computa los gradientes hacia atrás (backpropagation)
            L.backward()
            
            # Descenso de gradiente para actualizar los parámetros
            optimizador.step()
            
            # Limpia los gradientes
            optimizador.zero_grad()
            
        tiempo_epochs += time.process_time() - inicio_epoch
        
        if e % reports_every == 0:
            # Calcula la certeza de las predicciones sobre todo el conjunto
            X = dataset.X.to(device)
            Y = dataset.Y.to(device)

            # Predice usando la red
            Y_PRED = red.forward(X)
            
            # Calcula la pérdida de todo el conjunto
            L_total = perdida(Y_PRED, Y)

            # Elige una clase dependiendo del valor de Y_PRED
            Y_PRED_BIN = (Y_PRED >= 0.5).float()

            correctos = torch.sum(Y_PRED_BIN == Y).item()
            acc = (correctos / N) * 100

            sys.stdout.write(
                f"Epoch:{e:03d} Acc:{acc:.2f} Loss:{L_total:.4f} Tiempo/epoch:{tiempo_epochs/e:.3f}s\n"
            )

In [10]:
N = 5000 # numero de ejemplos
f = 300 # numero de features
epochs = 100

loop_FFNN_pytorch_style(
    dataset,
    batch_size=32,  # 10
    d1=300,
    d2=500,
    N=N,
    epochs=epochs,
    run_in_GPU=True,
    lr=0.06,
)

Número de parámetros de la red: FFNN(
  (fc1): Linear(in_features=300, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=1, bias=True)
)


Epoch:001 Acc:48.78 Loss:0.9442 Tiempo/epoch:0.177s
Epoch:002 Acc:51.22 Loss:0.7029 Tiempo/epoch:0.165s
Epoch:003 Acc:51.22 Loss:0.8564 Tiempo/epoch:0.163s
Epoch:004 Acc:51.22 Loss:0.7401 Tiempo/epoch:0.160s
Epoch:005 Acc:48.78 Loss:0.6952 Tiempo/epoch:0.161s
Epoch:006 Acc:48.78 Loss:0.8941 Tiempo/epoch:0.157s
Epoch:007 Acc:51.30 Loss:0.6898 Tiempo/epoch:0.154s
Epoch:008 Acc:51.22 Loss:0.7423 Tiempo/epoch:0.151s
Epoch:009 Acc:48.78 Loss:1.0828 Tiempo/epoch:0.149s
Epoch:010 Acc:48.78 Loss:0.7200 Tiempo/epoch:0.148s
Epoch:011 Acc:55.46 Loss:0.6862 Tiempo/epoch:0.146s
Epoch:012 Acc:58.54 Loss:0.6837 Tiempo/epoch:0.146s
Epoch:013 Acc:51.22 Loss:0.7034 Tiempo/epoch:0.145s
Epoch:014 Acc:48.84 Loss:0.7164 Tiempo/epoch:0.144s
Epoch:015 Acc:51.30 Loss:0.6994 Tiempo/epoch:0.143s
Epoch:016 Acc:51.22 Loss:0.7597 Tiempo/epoch:0.143s
Epoch:017 Acc:48.78 Loss:0.9168 Tiempo/epoch:0.143s
Epoch:018 Acc:48.78 Loss:0.8181 Tiempo/epoch:0.142s
Epoch:019 Acc:49.00 Loss:0.7602 Tiempo/epoch:0.142s
Epoch:020 Ac